## Нужные в этом тяжёлом деле (и не очень) библиотеки и модули

In [1]:
import numpy as np
import random as rd 
import simpy
import pandas as pd
import scipy as sp
import statsmodels 

## Генерация случайной величины
Экспоненциальное распределение с вводимым параметром lambda

In [3]:
lambda_ = float(input())
print(np.log(rd.random()) / lambda_)

-1.0092754168692466


## Простой процесс на simpy

In [4]:
def car(env):
    while True:
        print(f'Start parking at {env.now}')
        parking_duration = 5
        yield env.timeout(parking_duration)

        print(f'Start driving at {env.now}')
        trip_duration = 2
        yield env.timeout(trip_duration)

env = simpy.Environment()
env.process(car(env))
env.run(until=15)

Start parking at 0
Start driving at 5
Start parking at 7
Start driving at 12
Start parking at 14


## Запуск подпроцесса charge после парковки

In [7]:
class Car(object):
    def __init__(self, env):
        self.env = env
        # Start the run process everytime an instance is created.
        self.action = env.process(self.run())

    def run(self):
        while True:
            print(f'{id(self)} Start parking and charging at {self.env.now}')
            charge_duration = 5
            # We yield the process that process() returns
            # to wait for it to finish
            yield self.env.process(self.charge(charge_duration))

            # The charge process has finished and
            # we can start driving again.
            print(f'{id(self)} Start driving at {self.env.now}')
            trip_duration = 2
            yield self.env.timeout(trip_duration)

    def charge(self, duration):
        yield self.env.timeout(duration)

env = simpy.Environment()
car_1 = Car(env)
car_2 = Car(env)
env.run(until=15)

2391380788896 Start parking and charging at 0
2391380787648 Start parking and charging at 0
2391380788896 Start driving at 5
2391380787648 Start driving at 5
2391380788896 Start parking and charging at 7
2391380787648 Start parking and charging at 7
2391380788896 Start driving at 12
2391380787648 Start driving at 12
2391380788896 Start parking and charging at 14
2391380787648 Start parking and charging at 14


## Прерывание другого процесса

In [10]:
class Car(object):
    def __init__(self, env):
        self.env = env
        # Start the run process everytime an instance is created.
        self.action = env.process(self.run())

    def run(self):
        while True:
            print(f'{id(self)} Start parking and charging at {self.env.now}')
            charge_duration = 5
            # We may get interrupted while charging the battery
            try:
                yield self.env.process(self.charge(charge_duration))
            except simpy.Interrupt:
                # When we received an interrupt, we stop charging and
                # switch to the "driving" state
                print(f'{id(self)} Was interrupted. Hope, the battery is full enough ...')

            # The charge process has finished and
            # we can start driving again.
            print(f'{id(self)} Start driving at {self.env.now}')
            trip_duration = 2
            yield self.env.timeout(trip_duration)

    def charge(self, duration):
        yield self.env.timeout(duration)


def driver(env, car):
    yield env.timeout(3)
    car.action.interrupt()

env = simpy.Environment()
car_1 = Car(env)
car_2 = Car(env)
env.process(driver(env, car_1))
env.run(until=15)

2390860442352 Start parking and charging at 0
2391144724560 Start parking and charging at 0
2390860442352 Was interrupted. Hope, the battery is full enough ...
2390860442352 Start driving at 3
2390860442352 Start parking and charging at 5
2391144724560 Start driving at 5
2391144724560 Start parking and charging at 7
2390860442352 Start driving at 10
2390860442352 Start parking and charging at 12
2391144724560 Start driving at 12
2391144724560 Start parking and charging at 14


## Использование ресурсов
Добавляется устройство обслуживания с ёмкостью 2.
Машины 1 и 2 начинают обслуживаться сразу же по прибытии, а 3 и 4 ожидают освобождения слотов для обслуживания.

In [11]:
def car(env, name, bcs, driving_time, charge_duration):
    # Simulate driving to the BCS
    yield env.timeout(driving_time)
    # Request one of its charging spots
    print(f'{name} arriving at {env.now}')
    with bcs.request() as req:
        yield req
        # Charge the battery
        print(f'{name} starting to charge at {env.now}')
        yield env.timeout(charge_duration)
        print(f'{name} leaving the bcs at {env.now}')

env = simpy.Environment()
bcs = simpy.Resource(env, capacity=2)
for i in range(4):
    env.process(car(env, f'Car {i+1}', bcs, i*2, 5))
env.run()

Car 1 arriving at 0
Car 1 starting to charge at 0
Car 2 arriving at 2
Car 2 starting to charge at 2
Car 3 arriving at 4
Car 1 leaving the bcs at 5
Car 3 starting to charge at 5
Car 4 arriving at 6
Car 2 leaving the bcs at 7
Car 4 starting to charge at 7
Car 3 leaving the bcs at 10
Car 4 leaving the bcs at 12
